# New Section

In [49]:
import pandas as pd
import requests
import sqlite3
import json
from bs4 import BeautifulSoup
import sklearn.linear_model

## USED CHATGPT FOR CODING AID AND BRAINSTORMING IN ORDER TO COMPLETE THIS PROJECT

# URLs for datascraping
url = 'https://www.nfl.com/stats/player-stats/'
url2 = 'https://www.nfl.com/stats/player-stats/category/passing/2024/REG/all/passingyards/DESC?aftercursor=AAAAGQAAABlAoMIAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXlNVFExSWl3aU16SXdNRFF6TkRFdE5USXpOUzA0TnprM0xUTmlPV0V0TUdReU56RXhZekJpWXpJM0lpd2lNakF5TkNKZGZRPT0='

# Dictionary so we can merge the datasets based on teams
player_to_team = {
    "Joe Burrow": "Cincinnati Bengals", "Jared Goff": "Detroit Lions", "Baker Mayfield": "Tampa Bay Buccaneers",
    "Geno Smith": "Seattle Seahawks", "Sam Darnold": "Minnesota Vikings", "Lamar Jackson": "Baltimore Ravens",
    "Patrick Mahomes": "Kansas City Chiefs", "Aaron Rodgers": "New York Jets", "Justin Herbert": "Los Angeles Chargers",
    "Brock Purdy": "San Francisco 49ers", "Kyler Murray": "Arizona Cardinals", "Bo Nix": "Denver Broncos",
    "Matthew Stafford": "Los Angeles Rams", "Josh Allen": "Buffalo Bills", "C.J. Stroud": "Houston Texans",
    "Jayden Daniels": "Washington Commanders", "Caleb Williams": "Chicago Bears", "Kirk Cousins": "Atlanta Falcons",
    "Jordan Love": "Green Bay Packers", "Jalen Hurts": "Philadelphia Eagles", "Tua Tagovailoa": "Miami Dolphins",
    "Russell Wilson": "Pittsburgh Steelers", "Bryce Young": "Carolina Panthers", "Derek Carr": "New Orleans Saints",
    "Jameis Winston": "Cleveland Browns", "Will Levis": "Tennessee Titans", "Daniel Jones": "New York Giants",
    "Trevor Lawrence": "Jacksonville Jaguars", "Gardner Minshew": "Las Vegas Raiders", "Dak Prescott": "Dallas Cowboys",
    "Cooper Rush": "Dallas Cowboys", "Anthony Richardson": "Indianapolis Colts", "Joe Flacco": "Indianapolis Colts",
    "Mac Jones": "Jacksonville Jaguars", "Aidan O'Connell": "Las Vegas Raiders", "Mason Rudolph": "Tennessee Titans",
    "Deshaun Watson": "Cleveland Browns", "Justin Fields": "Pittsburgh Steelers", "Drew Lock": "New York Giants",
    "Andy Dalton": "Carolina Panthers", "Tyler Huntley": "Baltimore Ravens", "Jacoby Brissett": "New England Patriots",
    "Malik Willis": "Green Bay Packers", "Desmond Ridder": "Atlanta Falcons", "Marcus Mariota": "Washington Commanders",
    "Drake Maye": "New England Patriots", "Spencer Rattler": "New Orleans Saints", "Michael Penix Jr.": "Atlanta Falcons",
    "Dorian Thompson-Robinson": "Cleveland Browns", "Joshua Dobbs": "Arizona Cardinals"
}

response1 = requests.get(url)
soup1 = BeautifulSoup(response1.text, 'html.parser')

response2 = requests.get(url2)
soup2 = BeautifulSoup(response2.text, 'html.parser')

def Top50_Qb_Stats():
    # Datascraped Selectors
    names_selector = '.d3-o-media-object--vertical-center'
    yard_selector = '.selected'
    tds_selector = 'td:nth-child(7)'
    p_rtg_selector = 'td:nth-child(9)'

    # Extract data
    names1 = soup1.select(names_selector)
    yards1 = soup1.select(yard_selector)
    tds1 = soup1.select(tds_selector)
    p_rtgs1 = soup1.select(p_rtg_selector)

    names2 = soup2.select(names_selector)
    yards2 = soup2.select(yard_selector)
    tds2 = soup2.select(tds_selector)
    p_rtgs2 = soup2.select(p_rtg_selector)

    # Extract text and clean up
    names_list = [name.text.strip() for name in names1 + names2]
    yards_list = [int(yard.text.strip().replace(",", "")) if yard.text.strip() else 0 for yard in yards1 + yards2]
    tds_list = [int(td.text.strip().replace(",", "")) if td.text.strip() else 0 for td in tds1 + tds2]
    p_rtg_list = [float(p_rtg.text.strip()) if p_rtg.text.strip() else 0.0 for p_rtg in p_rtgs1 + p_rtgs2]

    # Create a DataFrame with player names, yardage, touchdowns, and passer rating
    df = pd.DataFrame({
        'Player Name': names_list,
        'Yards': yards_list,
        'TDS': tds_list,
        'Passer Rating': p_rtg_list
    })

    return df


def file_transformationCSV(file_name):
    user_response = input("Choose File Format ('Json', 'csv', 'sql_db'): ").lower()
    #CSV OPTION
    if user_response == "csv":
        print("User already has CSV file format.")
    #JSON OPTION
    elif user_response == "json":
        json_file = input("Enter the name of the output JSON file (e.g., output.json): ")
        df = pd.read_csv(file_name)  # Read the CSV file into a DataFrame
        # Convert DataFrame to JSON and save it
        df.to_json(json_file, lines=False)
        print(f"CSV data converted to JSON and saved to {json_file}")
    #SQL OPTION
    elif user_response == "sql_db":
        db_file = input("Enter the name of the output SQLite DB file (e.g., output.db): ")
        table_name = input("Enter the table name for the SQL conversion: ")
        df = pd.read_csv(file_name)
        conn = sqlite3.connect(db_file)
        df.to_sql(table_name, conn, if_exists='replace', index=False)
        conn.commit()
        conn.close()
        print(f"CSV data converted to SQL and saved to table '{table_name}' in {db_file}")
    else:
        print("Invalid Response. Please choose from 'csv', 'json', or 'sql_db'.")


def file_transformationJson(file_name):
    user_response = input("Choose File Format ('Json', 'csv', 'sql_db'): ").lower()
    #CSV OPTION
    if user_response == "csv":
        csv_file = input("Enter the name of the output CSV file (e.g., output.CSV): ")
        df = pd.read_json(file_name)
        df.to_csv(csv_file, index=False)
        print(f"Data saved to {csv_file} in CSV format.")
    #JSON OPTION
    elif user_response == "json":
        print("User already has JSON file")
    #SQL OPTION
    elif user_response == "sql_db":
        db_file = input("Enter the name of the output SQLite DB file (e.g., output.db): ")
        table_name = input("Enter the table name for the SQL conversion: ")
        df = pd.read_json(file_name)
        conn = sqlite3.connect(db_file)
        df.to_sql(table_name, conn, if_exists='replace', index=False)
        conn.commit()
        conn.close()
        print(f"Data saved to table '{table_name}' in database '{db_file}'.")

#Function to fetch table names from SQL database
def get_table_names(db_file):
    """
    Fetches all table names from the SQLite database.
    """
    conn = sqlite3.connect(db_file)
    query = "SELECT name FROM sqlite_master WHERE type='table';"
    tables = pd.read_sql(query, conn)
    conn.close()
    return tables['name'].tolist()


def file_transformationSQL_db(file_name):
    user_response = input("Choose File Format ('csv', 'json', 'sql_db'): ").lower()
    try:
        #CSV OPTION
        if user_response == "csv":
            # Read from SQLite database
            conn = sqlite3.connect(file_name)
            tables = get_table_names(file_name)
            if tables:
                print(f"Available tables in the database: {tables}")
                table_choice = int(input("Choose a table by number: ")) - 1
                table_name = tables[table_choice]
                df = pd.read_sql(f"SELECT * FROM {table_name}", conn)
                print(f"SQL data successfully loaded from table {table_name}.")
                conn.close()

                # Convert SQL data to CSV
                csv_file = input("Enter the name of the output CSV file (e.g., output.csv): ")
                df.to_csv(csv_file, index=False)
                print(f"Data saved to {csv_file} in CSV format.")
            else:
                print(f"No tables found in the database {file_name}.")
                conn.close()
                return
        #JSON OPTION
        elif user_response == "json":
            # Read from SQLite database
            conn = sqlite3.connect(file_name)
            tables = get_table_names(file_name)
            if tables:
                print(f"Available tables in the database: {tables}")
                table_choice = int(input("Choose a table by number: ")) - 1
                table_name = tables[table_choice]
                df = pd.read_sql(f"SELECT * FROM {table_name}", conn)
                print(f"SQL data successfully loaded from table {table_name}.")
                conn.close()

                # Convert SQL data to JSON
                json_file = input("Enter the name of the output JSON file (e.g., output.json): ")
                df.to_json(json_file, orient='records', lines=True)
                print(f"Data saved to {json_file} in JSON format.")
            else:
                print(f"No tables found in the database {file_name}.")
                conn.close()
                return
        #SQL OPTION
        elif user_response == "sql_db":
            # Read from SQLite database and ask for new SQLite DB to save as
            print("User already has SQL file")
        else:
            print(f"No tables found in the database {file_name}.")
            conn.close()
            return

    except Exception as e:
        print(f"Error reading the file: {e}")


def merge_with_team_stats(qb_df):
    # Reading data
    team_df = pd.read_csv('Wintotals;Team - Sheet1 (4).csv')

    # Clean column names
    team_df.columns = team_df.columns.str.strip()
    qb_df.columns = qb_df.columns.str.strip()

    # Add a new column 'Team' to the qb_df using the player_to_team dictionary
    qb_df['Team'] = qb_df['Player Name'].map(player_to_team)

    # Merge the QB stats with the team stats on "Team Name"
    merged_df = pd.merge(qb_df, team_df, left_on="Team", right_on="Team Name", how="left")

    # Ensure that we are adding both 'Team Name' and 'Team Wins' as columns
    merged_df = merged_df[['Player Name', 'Yards', 'TDS', 'Passer Rating', 'Team', 'Team Wins']]

    return merged_df


# Call the function to display overall passer stats and merge with team stats
qb_df = Top50_Qb_Stats()
merged_df = merge_with_team_stats(qb_df)

#Turning my merged dataset into a csv file, then into a sql db
csv_filename = "merged_qb_stats.csv"  # You can change this to your desired file name
merged_df.to_csv(csv_filename, index=True)
#Transforming the CSV into sql databasew
file_transformationCSV("merged_qb_stats.csv")




# *** DATA SUMMARIZATION *** #
#  Ingested Data: 32 Records; 2 Columns (CSV FILE)
#  Ingested Data: 50 Records: 4 Columns (Datascraped)
#  Digested Data: 50 Records: 6 Columns

# *** DATA ANALYSIS *** #
# We can see from the merged_qb_stats.csv (attached) (table) that stats like yardage have major implications on team wins. 8 of the top ten qbs leading the league in yardage have their teams at a minimum of ten wins. Passer rating is equally as important, for qbs without a servicable passer rating (93) have little chance of improving their team situations



Choose File Format ('Json', 'csv', 'sql_db'): csv
User already has CSV file format.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')